* Spark написан на Scala
* большая часть кода на нём исполняется на JVM (Java Virtual Machine)
* для работы со Spark помимио Scala можно использовать
    * Java
    * Python
    * R
* в случае использования pyspark некоторая часть кода исполняет интерпретатором Python

In [3]:
# создать SparkSession довольно просто
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

По умолчанию посмотреть на происходящее в SparkSession можно по адресу http://localhost:4040

In [4]:
# мы в основном будем работать с DataFrame'ами
# их можно создавать из файлов на диске
text_file = spark.read.text("../Dockerfile")
print(type(text_file))
text_file.show()

<class 'pyspark.sql.dataframe.DataFrame'>
+--------------------+
|               value|
+--------------------+
|  FROM ubuntu:artful|
|# install all nee...|
|RUN apt-get updat...|
|    python3-pip g...|
|    pip3 install ...|
|    jupyterlab RI...|
|    pyarrow annoy...|
|# surprise won't ...|
|RUN pip3 install ...|
|RUN jupyter-nbext...|
|    jupyter-nbext...|
|      # get the data|
|       WORKDIR /data|
|RUN wget http://f...|
|    unzip ml-late...|
|    wget http://m...|
|    && tar -xvf l...|
|      # get the code|
|        COPY . /code|
|# prepare the env...|
+--------------------+
only showing top 20 rows



In [5]:
import pandas as pd

# pandas DataFrame можно преобразовать в Spark DataFrame
pandas_df = pd.DataFrame([1,])
print(pandas_df.head())
spark_df = spark.createDataFrame(pd.DataFrame([1,]))
spark_df.show()
# и наоборот
spark_df.toPandas()

   0
0  1
+---+
|  0|
+---+
|  1|
+---+



,0
0,1


In [6]:
# в Spark вычисления ленивые
# эта строчка ничего не делает

filtered_data = text_file.where(text_file.value.contains("ENV"))

In [7]:
# а вот тут уже происходят вычисления

filtered_data.count()

4

In [8]:
# в отличие от pandas в Spark DataFrame - это не объект в памяти
# а просто граф вычислений
# чтобы явно создать объект в памяти, нужно DataFrame закешировать

filtered_data.cache()

DataFrame[value: string]

In [9]:
# кеширование - тоже ленивая операция
# вот происходит чтение с диска, фильтрация  и кэширование
# а потом уже вычисление
filtered_data.count()

4

In [10]:
# а теперь уже только вычисление
filtered_data.count()

4

In [11]:
# освободить кэш можно так
spark.catalog.clearCache()

In [12]:
# завершаем SparkSession
spark.stop()